###  importing relevant libraries

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from collections import Counter
from tabulate import tabulate
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import numpy as np
%matplotlib inline
import warnings
import sklearn
import time
import os
warnings.filterwarnings("ignore")
from sklearn.metrics import confusion_matrix
#time.sleep(18000)

### Discovering Labels

In [3]:
def target_name(name):
    df = pd.read_csv(name,usecols=["Label"])
    df["Label"]=df["Label"].replace(renamer)
    df=df.dropna()
    target_names=sorted(list(df["Label"].unique()))
    return target_names

In [4]:
def folder(f_name): #this function creates a folder.
    try:
        if not os.path.exists(f_name):
            os.makedirs(f_name)
    except OSError:
        print ("Tthe folder could not be created!")

In [5]:
def find_the_way(path,file_format,con=""):
    files_add = []
    # r=root, d=directories, f = files
    for r, d, f in os.walk(path):
        for file in f:
            if file_format in file:
                if con in file:
                    files_add.append(os.path.join(r, file))  
            
    return files_add

In [6]:
renamer={}

In [7]:
altime=0
#def most_frequent(List): 
#    return max(set(List), key = List.count) 


 
def most_frequent(List):
    occurence_count = Counter(List)
    occurence_count={k: v for k, v in sorted(occurence_count.items(), key=lambda item: item[1],reverse=True)}
    big=list(occurence_count.values())
    big=big.count(big[0])
    return list(occurence_count.keys())[np.random.randint(big)]


def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i * k + min(i, m):(i + 1) * k + min(i + 1, m)] for i in range(n))
def create_exception(df): 
    exception_list=[]
    dominant_mac=[]
    for i in df['aggregated'].unique():
        k=df[df['aggregated']==i]
        for ii in ['MAC']:
            hist = {}
            for x in k[ii].values:
                hist[x] = hist.get(x, 0) + 1
            hist=dict(sorted(hist.items(), key=lambda item: item[1],reverse=True))
            temp=next(iter(hist))
            if temp not in dominant_mac:
                dominant_mac.append(temp)
            else:
                exception_list.append(temp)
    return exception_list




def merged(m_test,predict,step,mixed):
    second=time.time()
    mac_test=[]
    for q in m_test.index:
        mac_test.append(m_test[q])

    d_list=sorted(list(m_test.unique()))
    devices={}
    for q in d_list:
        devices[q]=[]    


    new_y=[0]*len(m_test)

    for q,qq in enumerate (mac_test):
        devices[qq].append(q)
    for q in devices:
        a = [devices[q][j:j + step] for j in range(0, len(devices[q]), step)]  
        for qq in a:
            step_list=[]
            for qqq in qq:
                step_list.append(predict[qqq])
            add=most_frequent(list(step_list))
            for qqq in qq:
                new_y[qqq]=add
    results=pd.DataFrame(m_test)
    results["aggregated"]=new_y
    results["normal"]=predict
    
    #MIXED METHOD
    if mixed:
        exception=create_exception(results)
        for q in exception:
            results.loc[results.MAC == q, 'aggregated'] = results['normal']

    return results["aggregated"].values,time.time()-second

## Calculation of evaluations

In [8]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict, target_names=target_names,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [9]:


def Diff(li1, li2):
    li_dif = [i for i in li1 + li2 if i not in li1 or i not in li2]
    return li_dif
 
# Driver Code
#li1 = [10, 15, 20, 25, 30, 35, 40]
#li2 = [25, 40, 35]


In [10]:
def ML(loop1,loop2,output_csv1,cols,step,mixed,dname):

    
    for ii in ml_list:

        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
        output_csv=output_csv1.replace("ML",ii)
        if ii in ["SVM","LR"]:
            repetition=1
        else:
            repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]
        for i in range(repetition):



            #TRAIN
            df = pd.read_csv(loop1,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(loop2,usecols=cols)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes


            results_y=[]
            cv+=1
            results_y.append(y_test)


            #machine learning algorithm is applied in this section
            clf = ml_list[ii]#choose algorithm from ml_list dictionary
            second=time.time()
            clf.fit(X_train, y_train)
            train_time=(float((time.time()-second)) )
            second=time.time()
            predict =clf.predict(X_test)
            test_time=(float((time.time()-second)) )
            if step==1:
                altime=0
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
            else:
                predict,altime=merged(m_test,predict,step,mixed)
                line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ii)
                
                
            lines.append(line)
            df_cm = pd.DataFrame(confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
            
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv.replace("ML",ii),index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()

            
            
            
            
            



# Machine learning applications 

In [11]:
feature_list=feature= ['HH_0.01_covariance_0_1',
  'HH_0.01_mean_0',
  'HH_0.01_radius_0_1',
  'HH_0.01_std_0',
  'HH_0.1_covariance_0_1',
  'HH_0.1_magnitude_0_1',
  'HH_0.1_std_0',
  'HH_0.1_weight_0',
  'HH_1_magnitude_0_1',
  'HH_1_pcc_0_1',
  'HH_1_radius_0_1',
  'HH_1_std_0',
  'HH_1_weight_0',
  'HH_3_covariance_0_1',
  'HH_3_magnitude_0_1',
  'HH_3_radius_0_1',
  'HH_3_std_0',
  'HH_3_weight_0',
  'HH_5_mean_0',
  'HH_5_pcc_0_1',
  'HH_5_std_0',
  'HH_5_weight_0',
  'HH_jit_0.01_std',
  'HH_jit_0.1_mean',
  'HH_jit_0.1_std',
  'HH_jit_1_mean',
  'HH_jit_1_std',
  'HH_jit_1_weight',
  'HH_jit_3_std',
  'HH_jit_5_std',
  'HpHp_0.01_magnitude_0_1',
  'HpHp_0.01_radius_0_1',
  'HpHp_0.01_weight_0',
  'HpHp_0.1_covariance_0_1',
  'HpHp_0.1_magnitude_0_1',
  'HpHp_0.1_mean_0',
  'HpHp_0.1_pcc_0_1',
  'HpHp_0.1_std_0',
  'HpHp_0.1_weight_0',
  'HpHp_1_covariance_0_1',
  'HpHp_1_magnitude_0_1',
  'HpHp_1_mean_0',
  'HpHp_1_pcc_0_1',
  'HpHp_1_radius_0_1',
  'HpHp_1_std_0',
  'HpHp_3_pcc_0_1',
  'HpHp_3_radius_0_1',
  'HpHp_5_pcc_0_1',
  'HpHp_5_radius_0_1',
  'HpHp_5_std_0',
  'HpHp_5_weight_0',
  'MI_dir_0.01_mean',
  'MI_dir_0.01_std',
  'MI_dir_0.01_weight',
  'MI_dir_0.1_mean',
  'MI_dir_0.1_std',
  'MI_dir_0.1_weight',
  'MI_dir_1_std',
  'MI_dir_1_weight',
  'MI_dir_3_mean',
  'MI_dir_5_mean',
  'MI_dir_5_std',
  'MI_dir_5_weight',
  'Label']

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import xgboost

In [13]:

ml_list={"DT":DecisionTreeClassifier(criterion="entropy",max_depth=16,max_features=33,min_samples_split=5 ),           
"RF":RandomForestClassifier(bootstrap=False,criterion="entropy",max_depth=20,max_features=2,min_samples_split=2,n_estimators=79),    
"XGB":xgboost.XGBClassifier(n_estimators=1100,min_child_weight=1,max_depth=2,learning_rate=0.2),              
"KNN":KNeighborsClassifier(algorithm="ball_tree",leaf_size=44,n_neighbors=2,weights="distance"),
"NB":GaussianNB(var_smoothing=8.11131e-09), 
"LR":LogisticRegression(C= 0.071026 ,penalty="l1",solver="liblinear" ),"SVM":SVC(gamma=0.001,C=10)}


In [14]:
find_the_way("./","clean")

[]

In [15]:
file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [16]:
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/ML@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        dname=(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        print(dname)
        target_names=target_name(test)
        ML(train,test,output_csv,feature,step,mixed,dname)   



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  DT    0.494    0.511   0.513  0.511  0.486  0.468      0    5.85      0.016          0
 1  AD-S1-DI-S1    1     2  DT    0.457    0.476   0.487  0.476  0.446  0.43       0    6.062     0.031          0
 2  AD-S1-DI-S1    2     3  DT    0.453    0.47    0.492  0.47   0.439  0.426      0    5.662     0.016          0
 3  AD-S1-DI-S1    3     4  DT    0.456    0.474   0.481  0.474  0.446  0.428      0    6.077     0.022          0
 4  AD-S1-DI-S1    4     5  DT    0.469    0.486   0.524  0.486  0.471  0.441      0    6.127     0.016          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo          

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  NB    0.149    0.143   0.221  0.143  0.106  0.105      0    0.058     0.596          0
 1  AD-S1-DI-S1    1     2  NB    0.149    0.143   0.221  0.143  0.106  0.105      0    0.047     0.516          0
 2  AD-S1-DI-S1    2     3  NB    0.149    0.143   0.221  0.143  0.106  0.105      0    0.046     0.503          0
 3  AD-S1-DI-S1    3     4  NB    0.149    0.143   0.221  0.143  0.106  0.105      0    0.047     0.534          0
 4  AD-S1-DI-S1    4     5  NB    0.149    0.143   0.221  0.143  0.106  0.105      0    0.047     0.532          0

                                  precision    recall    f1-score      support
------------------------------  -----------  --------  ----------  -----------
Amazon Echo                     0.08

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S1    0     1  RF    0.627    0.637   0.704  0.637  0.626  0.608      0   36.485     1.025          0
 1  DI-S1-AD-S1    1     2  RF    0.632    0.641   0.724  0.641  0.633  0.613      0   36.488     1.048          0
 2  DI-S1-AD-S1    2     3  RF    0.634    0.644   0.737  0.644  0.642  0.615      0   36.709     1.024          0
 3  DI-S1-AD-S1    3     4  RF    0.63     0.641   0.718  0.641  0.636  0.611      0   35.002     0.871          0
 4  DI-S1-AD-S1    4     5  RF    0.626    0.636   0.722  0.636  0.634  0.607      0   30.708     0.868          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S1    0     1  LR    0.413    0.422   0.459  0.422  0.412  0.383      0  1649.42     0.016          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                        0.481236  0.436       0.457503   1000
Awair air quality monitor          0.430328  0.315       0.363741   1000
Belkin Wemo switch                 0.611755  0.739       0.669384   1000
Belkin wemo motion sensor          0.553444  0.233       0.327938   1000
Dropcam                            0.623922  0.579       0.600622   1000
HP Printer                         0.393103  0.114       0.176744   1000
Light Bulbs LiFX Smart Bulb        0.293255  0.3        

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S2    0     1  XGB    0.67    0.676   0.684  0.676  0.649  0.653      0  194.518     2.909          0
 1  AD-S1-DI-S2    1     2  XGB    0.67    0.676   0.684  0.676  0.649  0.653      0  184.411     2.904          0
 2  AD-S1-DI-S2    2     3  XGB    0.67    0.676   0.684  0.676  0.649  0.653      0  200.13      3.528          0
 3  AD-S1-DI-S2    3     4  XGB    0.67    0.676   0.684  0.676  0.649  0.653      0  189.098     2.889          0
 4  AD-S1-DI-S2    4     5  XGB    0.67    0.676   0.684  0.676  0.649  0.653      0  194.864     2.841          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

...

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S2    0     1  NB    0.187    0.173   0.227  0.173  0.125  0.142      0    0.038     0.401          0
 1  DI-S2-AD-S2    1     2  NB    0.187    0.173   0.227  0.173  0.125  0.142      0    0.038     0.379          0
 2  DI-S2-AD-S2    2     3  NB    0.187    0.173   0.227  0.173  0.125  0.142      0    0.047     0.385          0
 3  DI-S2-AD-S2    3     4  NB    0.187    0.173   0.227  0.173  0.125  0.142      0    0.031     0.401          0
 4  DI-S2-AD-S2    4     5  NB    0.187    0.173   0.227  0.173  0.125  0.142      0    0.038     0.423          0

                                  precision      recall    f1-score       support
------------------------------  -----------  ----------  ----------  ------------
Amazon Echo                   

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  RF    0.69     0.7     0.685  0.7    0.663  0.674      0   24.186     0.887          0
 1  AD-S2-DI-S1    1     2  RF    0.69     0.7     0.7    0.7    0.671  0.674      0   25.818     0.886          0
 2  AD-S2-DI-S1    2     3  RF    0.673    0.685   0.672  0.685  0.647  0.657      0   25.443     0.864          0
 3  AD-S2-DI-S1    3     4  RF    0.671    0.683   0.664  0.683  0.645  0.655      0   25.795     0.864          0
 4  AD-S2-DI-S1    4     5  RF    0.697    0.708   0.711  0.708  0.68   0.682      0   25.132     0.871          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  LR    0.408    0.422   0.446  0.422  0.393  0.378      0  2420.76     0.016          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                        0.227022  0.536      0.318953    1000
Awair air quality monitor          0.470721  0.418      0.442797    1000
Belkin Wemo switch                 0.479381  0.093      0.155779    1000
Belkin wemo motion sensor          0.147609  0.071      0.0958812   1000
Dropcam                            0.893813  0.968      0.929429    1000
HP Printer                         0.333056  0.4        0.363471    1000
Light Bulbs LiFX Smart Bulb        0.202367  0.171      

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  XGB   0.747    0.749   0.755  0.749  0.737  0.734      0  187.912     2.839          0
 1  DI-S1-AD-S2    1     2  XGB   0.747    0.749   0.755  0.749  0.737  0.734      0  207.603     2.994          0
 2  DI-S1-AD-S2    2     3  XGB   0.747    0.749   0.755  0.749  0.737  0.734      0  188.693     3.017          0
 3  DI-S1-AD-S2    3     4  XGB   0.747    0.749   0.755  0.749  0.737  0.734      0  200.566     2.924          0
 4  DI-S1-AD-S2    4     5  XGB   0.747    0.749   0.755  0.749  0.737  0.734      0  186.992     2.975          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S1-AD-S2    0     1  LR    0.498    0.521    0.52  0.521  0.488  0.472      0  1562.23     0.016          0

                                  precision    recall    f1-score       support
------------------------------  -----------  --------  ----------  ------------
Amazon Echo                       0.542992   0.461      0.498648    1000
Awair air quality monitor         0.408654   0.34       0.371179    1000
Belkin Wemo switch                0.876068   0.82       0.847107    1000
Belkin wemo motion sensor         0.728337   0.311      0.435879    1000
Dropcam                           0.570922   0.966      0.717682    1000
HP Printer                        0.259036   0.086      0.129129    1000
Light Bulbs LiFX Smart Bulb       0.176268   0.221382   

In [17]:
#!shutdown /s /t 360

# MLP

In [18]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
def create_model(params):
    model = Sequential()

    # Giriş katmanı
    model.add(Dense(units=params['units_input'], activation='relu', input_shape=(input_shape,)))

    # Gizli katmanlar
    for i in range(params['num_layers']):
        model.add(Dense(units=params[f'units_{i}'], activation='relu'))
        model.add(Dropout(params[f'dropout_{i}']))

    # Çıkış katmanı
    model.add(Dense(num_classes, activation='softmax'))

    # Optimizasyon ve kayıp fonksiyonunun belirlenmesi
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {'units_input': 32, 'num_layers': 1, 'units_0': 32, 'dropout_0': 0.0, 
               'learning_rate': 0.001, 'units_1': 32, 'dropout_1': 0.2, 'units_2': 128, 'dropout_2': 0.4}
say=0
dataset="./results/other/1/"
folder(dataset)

for ii in file_list:
        print()
        say+=1
        train=file_list[ii][0]
        test=file_list[ii][1]
        mixed=False
        step=1
        sayac=1
        output_csv=f"{dataset}/MLP@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
        print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
        target_names=target_name(test)


        class_based_results=pd.DataFrame()#"" #pd.DataFrame(0, index=np.arange((len(target_names)+3)), columns=["f1-score","precision","recall","support"])
        cm=pd.DataFrame()
        cv=0
    
        repetition=5
        lines=[["Dataset","T","CV","ML","Acc","b_Acc","Prec","Rec","F1","kap","ROC","tra-T","test-T","Al-Time"]]


    
        for i in range(repetition):

            df = pd.read_csv(train,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(renamer)
            X_train =df[df.columns[0:-1]]
            X_train=np.array(X_train)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_train=df[df.columns[-1]].cat.codes  
            
            

            #TEST
            df = pd.read_csv(test,usecols=feature)
            df=df.dropna()
            df["Label"]=df["Label"].replace(feature)
            df = shuffle(df, random_state=42)

            X_test =df[df.columns[0:-1]]
            X_test=np.array(X_test)
            df[df.columns[-1]] = df[df.columns[-1]].astype('category')
            y_test=df[df.columns[-1]].cat.codes
            
            y_train = to_categorical(y_train)
            #y_test = to_categorical(y_test)
            min_max_scaler = preprocessing.MinMaxScaler()
            X_train = min_max_scaler.fit_transform(X_train)
            X_test = min_max_scaler.fit_transform(X_test)
            
            input_shape=X_train.shape[1]
            num_classes=len(y_test.unique())
            # Modelin oluşturulması

            results_y=[]
            cv+=1
            results_y.append(y_test)

            model = create_model(best_params)
            #model.summary()
            second=time.time()
            model = create_model(best_params)
            model.fit(X_train, y_train, epochs=10, batch_size=32,validation_split=0.3,verbose= 0)#validation_data=(X_test, y_test)) #validation_split=0.3)
            train_time=(float((time.time()-second)) )
            second=time.time()
            
            predict =model.predict(X_test)
            predict = np.argmax(predict, axis=1)
            test_time=(float((time.time()-second)) )
            
            
            altime=0
            dname=f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
            line,class_based_results=score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,"MLP")
            
                
                
              
            lines.append(line)
            df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
            if cm.empty:
                cm =df_cm
            else:
                cm = cm.add(df_cm, fill_value=0)
                        
        class_based_results=class_based_results/(repetition)
        results = pd.DataFrame (lines[1:], columns = lines[0])
        results.to_csv(output_csv,index=False)
        results=results.round(3)
        print (tabulate(results, headers=list(results.columns)))
        print()
  
        
        print (tabulate(class_based_results, headers=list(class_based_results.columns)))
        class_based_results.to_csv(output_csv.replace(".csv","CB.csv"))
        if True:
            cm=cm//repetition
            cm.to_csv(output_csv.replace(".csv","CM.csv"))
            #graph_name=output_csv+"_confusion matrix.pdf"   
            #plt.figure(figsize = (40,28))
            #sns.heatmap(cm,xticklabels=target_names, yticklabels=target_names, annot=True, fmt='g')
            #plt.savefig(graph_name,bbox_inches='tight')#, dpi=400)
            #plt.show()
            #print(cm)
            #print("\n\n\n")             


            #plt.clf()
  


AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  MLP   0.102    0.099   0.105  0.099  0.076  0.056      0   11.512     0.842          0
 1  AD-S1-DI-S1    1     2  MLP   0.102    0.098   0.072  0.098  0.06   0.055      0   10.587     0.771          0
 2  AD-S1-DI-S1    2     3  MLP   0.11     0.106   0.055  0.106  0.06   0.064      0   10.438     0.764          0
 3  AD-S1-DI-S1    3     4  MLP   0.074    0.071   0.114  0.071  0.046  0.026      0   10.509     1.465          0
 4  AD-S1-DI-S1    4     5  MLP   0.098    0.095   0.12   0.095  0.061  0.052      0   11.003     0.764          0

                                  precision     recall     f1-score        support
------------------------------  -----------  ---------  -----------  -------------
Amazon Echo    

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  MLP   0.113    0.109   0.071  0.109  0.062  0.067      0   10.524     0.765          0
 1  DI-S2-AD-S1    1     2  MLP   0.127    0.122   0.102  0.122  0.083  0.081      0   11.595     0.795          0
 2  DI-S2-AD-S1    2     3  MLP   0.126    0.122   0.077  0.122  0.079  0.081      0   10.594     0.77           0
 3  DI-S2-AD-S1    3     4  MLP   0.123    0.119   0.143  0.119  0.066  0.078      0   10.641     0.771          0
 4  DI-S2-AD-S1    4     5  MLP   0.134    0.13    0.13   0.13   0.076  0.089      0   11.172     0.77           0

                                  precision       recall    f1-score       support
------------------------------  -----------  -----------  ----------  ------------
Amazon Echo                 

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  MLP   0.111    0.107   0.109  0.107  0.064  0.065      0    8.821     0.702          0
 1  AD-S2-DI-S1    1     2  MLP   0.069    0.066   0.111  0.066  0.039  0.02       0    8.751     1.018          0
 2  AD-S2-DI-S1    2     3  MLP   0.089    0.085   0.087  0.085  0.046  0.041      0    9.407     0.755          0
 3  AD-S2-DI-S1    3     4  MLP   0.116    0.113   0.071  0.113  0.061  0.072      0    9.78      0.771          0
 4  AD-S2-DI-S1    4     5  MLP   0.092    0.089   0.092  0.089  0.065  0.045      0    9.706     0.793          0

                                  precision      recall    f1-score        support
------------------------------  -----------  ----------  ----------  -------------
Amazon Echo                 

# CNN

In [19]:
import sklearn
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing

def create_model(params):
    model = Sequential()

    # Convolutional layer
    model.add(Conv1D(filters=params['conv1d_filters'], kernel_size=params['conv1d_kernel_size'], activation='relu', input_shape=((X_train.shape[1], X_train.shape[2]))))

    # Pooling layer
    model.add(MaxPooling1D(pool_size=params['max_pooling1d_pool_size']))

    # Flatten layer
    model.add(Flatten())

    # Dense layers
    model.add(Dense(units=params['dense_units_1'], activation='relu'))
    model.add(Dropout(params['dropout_rate_1']))
    model.add(Dense(units=params['dense_units_2'], activation='relu'))
   # model.add(Dropout(params['dropout_rate_2']))
    model.add(Dense(units=21, activation='softmax'))

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=params['learning_rate'])
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

best_params = {
    "conv1d_filters": 32,
    "conv1d_kernel_size": 3,
    "max_pooling1d_pool_size": 2,
    "dense_units_1": 32,
    "dropout_rate_1": 0.0,
    "dense_units_2": 21,
    "learning_rate": 0.01
}


say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/CNN@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        df = pd.read_csv(train, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(renamer)
        X_train = df[df.columns[0:-1]]
        X_train = np.array(X_train)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_train = df[df.columns[-1]].cat.codes  

        df = pd.read_csv(test, usecols=feature)
        df = df.dropna()
        df["Label"] = df["Label"].replace(feature)
        df = shuffle(df, random_state=42)

        X_test = df[df.columns[0:-1]]
        X_test = np.array(X_test)
        df[df.columns[-1]] = df[df.columns[-1]].astype('category')
        y_test = df[df.columns[-1]].cat.codes

        y_train = to_categorical(y_train)
        min_max_scaler = preprocessing.MinMaxScaler()
        X_train = min_max_scaler.fit_transform(X_train)
        X_test = min_max_scaler.fit_transform(X_test)
        # Veriyi CNN modeline uygun hale getirme
        X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        input_shape = X_train.shape[1]
        num_classes = len(y_test.unique())

        results_y = []
        cv += 1
        results_y.append(y_test)

        model = create_model(best_params)
        second = time.time()
        model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "CNN")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     1  CNN   0.101    0.098   0.099  0.098  0.067  0.054      0   21.295     1.334          0
 1  AD-S1-DI-S1    1     2  CNN   0.091    0.088   0.06   0.088  0.046  0.044      0   21.801     1.35           0
 2  AD-S1-DI-S1    2     3  CNN   0.106    0.102   0.112  0.102  0.067  0.06       0   21.412     1.341          0
 3  AD-S1-DI-S1    3     4  CNN   0.108    0.104   0.077  0.104  0.068  0.062      0   21.38      1.34           0
 4  AD-S1-DI-S1    4     5  CNN   0.151    0.146   0.104  0.146  0.1    0.108      0   21.264     1.35           0

                                  precision      recall    f1-score       support
------------------------------  -----------  ----------  ----------  ------------
Amazon Echo      

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     1  CNN   0.1      0.096   0.064  0.096  0.051  0.054      0   20.18      1.24           0
 1  DI-S2-AD-S1    1     2  CNN   0.109    0.117   0.069  0.117  0.065  0.065      0   19.713     1.219          0
 2  DI-S2-AD-S1    2     3  CNN   0.135    0.13    0.096  0.13   0.089  0.091      0   22.469     1.319          0
 3  DI-S2-AD-S1    3     4  CNN   0.068    0.066   0.146  0.066  0.05   0.023      0   20.869     1.72           0
 4  DI-S2-AD-S1    4     5  CNN   0.099    0.095   0.08   0.095  0.054  0.055      0   21.481     1.341          0

                                  precision     recall     f1-score       support
------------------------------  -----------  ---------  -----------  ------------
Amazon Echo                   

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S1    0     1  CNN   0.078    0.075   0.118  0.075  0.04   0.03       0   20.146     1.381          0
 1  AD-S2-DI-S1    1     2  CNN   0.07     0.067   0.069  0.067  0.034  0.022      0   19.978     1.435          0
 2  AD-S2-DI-S1    2     3  CNN   0.082    0.093   0.087  0.093  0.06   0.035      0   19.744     1.334          0
 3  AD-S2-DI-S1    3     4  CNN   0.093    0.089   0.113  0.089  0.061  0.046      0   19.746     1.466          0
 4  AD-S2-DI-S1    4     5  CNN   0.094    0.091   0.081  0.091  0.058  0.047      0   19.824     1.356          0

                                  precision     recall     f1-score        support
------------------------------  -----------  ---------  -----------  -------------
Amazon Echo                 

# LSTM

In [21]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import time

# Veri yükleme fonksiyonu
def load_data(file_path, feature):
    df = pd.read_csv(file_path, usecols=feature) 
    X = df.iloc[:, :-1]
    df['Label'] = df['Label'].astype('category')
    y = df['Label'].cat.codes  
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)
    X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], X_scaled.shape[1], 1))
    return X_reshaped, y

# Model oluşturma fonksiyonu
def create_model(params):
    model = Sequential()
    model.add(layers.LSTM(units=params['lstm_0_units'],
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params['dropout_1']))
    
    for i in range(1, params['num_layers']):
        model.add(layers.LSTM(units=params[f'lstm_{i}_units'], return_sequences=True))
        model.add(layers.Dropout(rate=params[f'dropout_{i+1}']))
    
    model.add(layers.LSTM(units=params['lstm_1_units']))
    model.add(layers.Dropout(rate=params['dropout_last']))
    
    model.add(layers.Dense(params['units_last'], activation='softmax'))
    return model

# Parametreler
params = {'units': 32, 'dropout_1': 0.1, 'num_layers': 1, 'lstm_0_units': 128, 'dropout_2': 0.4, 'units_last': 96,
          'dropout_last': 0.30000000000000004, 'learning_rate': 0.0001, 'lstm_1_units': 128, 'dropout_3': 0.4, 'lstm_2_units': 64, 'dropout_4': 0.0}

# Diğer değişkenler ve döngü
say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 5
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        X_train, y_train = load_data(file_list[ii][0], feature)
        X_test, y_test = load_data(file_list[ii][1], feature)
        
        model = create_model(params)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S1-DI-S1
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S1-DI-S1    0     0  LSTM  0.068    0.066   0.033  0.066  0.043  0.02       0  689.825    35.846          0
 1  AD-S1-DI-S1    1     0  LSTM  0.087    0.084   0.04   0.084  0.051  0.04       0  686.131    36.164          0
 2  AD-S1-DI-S1    2     0  LSTM  0.067    0.064   0.033  0.064  0.041  0.018      0  691.378    37.168          0
 3  AD-S1-DI-S1    3     0  LSTM  0.089    0.086   0.042  0.086  0.051  0.042      0  697.401    38.573          0
 4  AD-S1-DI-S1    4     0  LSTM  0.072    0.069   0.042  0.069  0.047  0.023      0  814.166    45.664          0

                                  precision     recall     f1-score        support
------------------------------  -----------  ---------  -----------  -------------
Amazon Echo    

    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1     kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  ------  -----  -------  --------  ---------
 0  DI-S2-AD-S1    0     0  LSTM  0.034    0.033   0.088  0.033  0.015  -0.016      0  966.454    51.766          0
 1  DI-S2-AD-S1    1     0  LSTM  0.074    0.072   0.016  0.072  0.026   0.026      0  863.927    51.847          0
 2  DI-S2-AD-S1    2     0  LSTM  0.055    0.053   0.015  0.053  0.017   0.006      0  822.577    50.626          0
 3  DI-S2-AD-S1    3     0  LSTM  0.064    0.062   0.017  0.062  0.019   0.015      0  931.454    54.26           0
 4  DI-S2-AD-S1    4     0  LSTM  0.046    0.044   0.025  0.044  0.02   -0.004      0  873.872    51.856          0

                                  precision     recall     f1-score       support
------------------------------  -----------  ---------  -----------  ------------
Amazon Echo            

ValueError: Number of classes, 29, does not match size of target_names, 21. Try specifying the labels parameter

In [30]:
file_list={#"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
#"DD11T":['./small/DI-S1.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
#"DD12":['./small/AD-S1.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
#"DD12T":['./small/DI-S2.csv','./small/AD-S1.csv','./small/AD-S2.csv'],
"DD22":['./small/AD-S2.csv','./small/DI-S2.csv','./small/DI-S1.csv'],
"DD22T":['./small/DI-S2.csv','./small/AD-S2.csv','./small/AD-S1.csv'],
"DD21":['./small/AD-S2.csv','./small/DI-S1.csv','./small/DI-S2.csv'],
"DD21T":['./small/DI-S1.csv','./small/AD-S2.csv','./small/AD-S1.csv']}
#file_list={"DD11":['./small/AD-S1.csv','./small/DI-S1.csv','./small/DI-S2.csv']}

In [31]:
def score(altime,train_time,test_time,predict,y_test,class_based_results,i,cv,dname,ml):
    rc=sklearn.metrics.recall_score(y_test, predict,average= "macro")
    pr=sklearn.metrics.precision_score(y_test, predict,average= "macro")
    f_1=sklearn.metrics.f1_score(y_test, predict,average= "macro")     
    accuracy=sklearn.metrics.accuracy_score(y_test, predict)
    accuracy_b=sklearn.metrics.balanced_accuracy_score( y_test,predict)
    kappa=sklearn.metrics.cohen_kappa_score(y_test, predict,labels=None, weights=None, sample_weight=None)
    try:
        roc=sklearn.metrics.roc_auc_score(y_test, predict)
    except:roc=0
    report = sklearn.metrics.classification_report(y_test, predict,output_dict=True)
    cr = pd.DataFrame(report).transpose()
    line=[dname,i,cv,ml,accuracy,accuracy_b,pr,rc,f_1,kappa,roc,train_time,test_time,altime]

    if class_based_results.empty:
        class_based_results =cr
    else:
        class_based_results = class_based_results.add(cr, fill_value=0)
    return line,class_based_results

In [33]:
import sklearn
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers 
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn import preprocessing
import time

# Veri yükleme fonksiyonu
def load_data(file_path, feature):
    df = pd.read_csv(file_path, usecols=feature) 
    X = df.iloc[:, :-1]
    df['Label'] = df['Label'].astype('category')
    y = df['Label'].cat.codes  
    min_max_scaler = preprocessing.MinMaxScaler()
    X_scaled = min_max_scaler.fit_transform(X)
    X_reshaped = np.reshape(X_scaled, (X_scaled.shape[0], X_scaled.shape[1], 1))
    return X_reshaped, y

# Model oluşturma fonksiyonu
def create_model(params):
    model = Sequential()
    model.add(layers.LSTM(units=params['lstm_0_units'],
                          return_sequences=True,
                          input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.Dropout(rate=params['dropout_1']))
    
    for i in range(1, params['num_layers']):
        model.add(layers.LSTM(units=params[f'lstm_{i}_units'], return_sequences=True))
        model.add(layers.Dropout(rate=params[f'dropout_{i+1}']))
    
    model.add(layers.LSTM(units=params['lstm_1_units']))
    model.add(layers.Dropout(rate=params['dropout_last']))
    
    model.add(layers.Dense(params['units_last'], activation='softmax'))
    return model

# Parametreler
params = {'units': 32, 'dropout_1': 0.1, 'num_layers': 1, 'lstm_0_units': 128, 'dropout_2': 0.4, 'units_last': 96,
          'dropout_last': 0.30000000000000004, 'learning_rate': 0.0001, 'lstm_1_units': 128, 'dropout_3': 0.4, 'lstm_2_units': 64, 'dropout_4': 0.0}

# Diğer değişkenler ve döngü
say = 0
dataset = "./results/other/1/"
folder(dataset)

for ii in file_list:
    print()
    say += 1
    train = file_list[ii][0]
    test = file_list[ii][1]
    mixed = False
    step = 1
    sayac = 1
    output_csv = f"{dataset}/LSTM@{file_list[ii][0][8:-4]}@{file_list[ii][1][8:-4]}.csv"
    print(f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}")
    target_names = target_name(test)

    class_based_results = pd.DataFrame()
    cm = pd.DataFrame()
    cv = 0
    repetition = 1
    lines = [["Dataset", "T", "CV", "ML", "Acc", "b_Acc", "Prec", "Rec", "F1", "kap", "ROC", "tra-T", "test-T", "Al-Time"]]

    for i in range(repetition):
        X_train, y_train = load_data(file_list[ii][0], feature)
        X_test, y_test = load_data(file_list[ii][1], feature)
        
        model = create_model(params)
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=params['learning_rate']),
                      loss='sparse_categorical_crossentropy',
                      metrics=["accuracy"])
        
        second = time.time()
        model.fit(X_train, y_train, epochs=10, validation_split=0.3, verbose=0)
        train_time = (float((time.time() - second)))
        second = time.time()

        predict = model.predict(X_test)
        predict = np.argmax(predict, axis=1)
        test_time = (float((time.time() - second)))

        altime = 0
        dname = f"{file_list[ii][0][8:-4]}-{file_list[ii][1][8:-4]}"
        line, class_based_results = score(altime, train_time, test_time, predict, y_test, class_based_results, i, cv, dname, "LSTM")

        lines.append(line)
        df_cm = pd.DataFrame(sklearn.metrics.confusion_matrix(y_test, predict))
        if cm.empty:
            cm = df_cm
        else:
            cm = cm.add(df_cm, fill_value=0)

    class_based_results = class_based_results / repetition
    results = pd.DataFrame(lines[1:], columns=lines[0])
    results.to_csv(output_csv, index=False)
    results = results.round(3)
    print(tabulate(results, headers=list(results.columns)))
    print()

    print(tabulate(class_based_results, headers=list(class_based_results.columns)))
    class_based_results.to_csv(output_csv.replace(".csv", "CB.csv"))
    if True:
        cm = cm // repetition
        cm.to_csv(output_csv.replace(".csv", "CM.csv"))



AD-S2-DI-S2
637/637 ━━━━━━━━━━━━━━━━━━━━ 32s 50ms/step
    Dataset        T    CV  ML      Acc    b_Acc    Prec    Rec     F1    kap    ROC    tra-T    test-T    Al-Time
--  -----------  ---  ----  ----  -----  -------  ------  -----  -----  -----  -----  -------  --------  ---------
 0  AD-S2-DI-S2    0     0  LSTM  0.077    0.074   0.057  0.074  0.044  0.029      0  547.935    41.177          0

                precision     recall    f1-score        support
------------  -----------  ---------  ----------  -------------
0               0.0105263  0.003      0.00466926   1000
1               0.003125   0.004      0.00350877   1000
2               0.0440529  0.01       0.0162999    1000
3               0.164021   0.031      0.0521447    1000
4               0.112254   0.131      0.120904     1000
5               0          0          0            1000
6               0          0          0            1000
7               0.109677   0.119      0.114149     1000
8               0     